In [1]:
import pandas as pd
data = pd.read_csv('./Databases/Australia Market _up to 2022_consumption.csv')
data = data.iloc[:,[2,3,4,8,11]]
data =data.loc[(data.Attribute_Description != "Beginning Stocks") &  (data.Attribute_Description != "Ending Stocks") &  (data.Attribute_Description != "Total Distribution") &  (data.Attribute_Description != "Total Supply"),: ]
data["Measure_in_tons"] = data.Value * 60
data =data.drop(columns=["Value"])
data = data.pivot_table('Measure_in_tons', ['Country_Code', 'Country_Name','Market_Year'], 'Attribute_Description')
data = data.reset_index()
data.rename(columns={"Domestic Consumption":"Total_consumption",
                     "Arabica Production":"Arabica_Production",
                     "Bean Exports":"Bean_Exports",
                     "Bean Imports":"Bean_Imports",
                     "Other Production":"Other_Production",
                     "Roast & Ground Exports":"Roast_Ground_Exports",
                     "Roast & Ground Imports":"Roast_Ground_Imports",
                     "Robusta Production":"Robusta_Production",
                     "Rst,Ground Dom. Consum":"Roast_Ground_consumption",
                     "Soluble Dom. Cons.":"Soluble_consumption",
                     "Soluble Exports":"Soluble_Exports",
                     "Soluble Imports":"Soluble_Imports",
                     "Exports":"Total_Exports",
                     "Imports":"Total_Imports",
                     "Production":"Total_Production"}, inplace=True)
data = data.loc[:,['Country_Code','Country_Name','Market_Year','Arabica_Production','Robusta_Production','Other_Production','Total_Production',
                   'Bean_Exports','Roast_Ground_Exports','Soluble_Exports','Total_Exports',
                   'Bean_Imports','Roast_Ground_Imports','Soluble_Imports','Total_Imports',
                   'Roast_Ground_consumption','Soluble_consumption','Total_consumption']]
data = data.loc[data.Market_Year>=2017,:]

data.to_csv("worldwide_coffee_market.csv",index=False)




In [7]:
# Load CSV file to Postgres database.
import numpy as np
from sqlalchemy import create_engine
data = pd.read_csv('worldwide_coffee_market.csv')
password = "postgres:password123456"
engine = create_engine(f'postgresql://{password}@localhost:5432/coffee')
data.to_sql(name='coffee_worldwide', con=engine, if_exists='append', index=False)

In [8]:
engine.table_names()

c:\Users\datqu\anaconda3\envs\bootcamp\lib\site-packages\ipykernel_launcher.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  """Entry point for launching an IPython kernel.


['coffee_worldwide']

In [9]:
pd.read_sql_query('select * from coffee_worldwide', con=engine).head()

,Country_Code,Country_Name,Market_Year,Arabica_Production,Robusta_Production,Other_Production,Total_Production,Bean_Exports,Roast_Ground_Exports,Soluble_Exports,Total_Exports,Bean_Imports,Roast_Ground_Imports,Soluble_Imports,Total_Imports,Roast_Ground_consumption,Soluble_consumption,Total_consumption
0,AG,Algeria,2017,0,0,0,0,0,0,0,0,129900,0,2400,132300,129900,2400,132300
1,AG,Algeria,2018,0,0,0,0,0,0,0,0,136200,0,1800,138000,136200,1800,138000
2,AG,Algeria,2019,0,0,0,0,0,0,0,0,138000,0,2400,140400,138000,2400,140400
3,AG,Algeria,2020,0,0,0,0,0,0,0,0,120000,0,2400,122400,120000,2400,122400
4,AG,Algeria,2021,0,0,0,0,0,0,0,0,132000,0,2400,134400,132000,2400,134400


In [2]:
# # Load CSV file to SQLITE database
# from pathlib import Path
# # create database file empty
# Path('db.sqlite').touch()
# import sqlite3
# # connect to database
# conn = sqlite3.connect('db.sqlite')
# c = conn.cursor()
# # add empty columns
# c.execute('''CREATE TABLE coffee (Country_Code text, Country_Name text, Market_Year int, Arabica_Production int, Robusta_Production int, Other_Production int,Total_Production int,Bean_Exports int,Roast_Ground_Exports int,Soluble_Exports int,Total_Exports int,Bean_Imports int,Roast_Ground_Imports int,Soluble_Imports int,Total_Imports int,Roast_Ground_consumption int,Soluble_consumption int,Total_consumption int)''')
# # read csv file
# data = pd.read_csv('worldwide_coffee_market.csv')
# # write all data to database
# data.to_sql('coffee', conn, if_exists='append', index = False)
